# Backtest

In [17]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

# Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')

# Select a subset of columns
dataset = dataset.loc[idx[:, '2005-02-15':'2023-02-15'], ['adjusted_close', 'volume']]

# Assign names to the index
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

# Convert the 'date' column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

class CustomPandasData(bt.feeds.PandasData):
    lines = ('close', 'volume')
    params = (
        ('datetime', 'date'),
        ('open', None),
        ('high', None),
        ('low', None),
        ('close', 'adjusted_close'),
        ('volume', 'volume'),
        ('openinterest', None)
    )

class SimpleMovingAverageCross(bt.Strategy):
    params = (
        ("fast", 10),
        ("slow", 30),
    )

    def __init__(self):
        self.fast_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)

    def next(self):
        if self.fast_sma[0] > self.slow_sma[0] and not self.position:
            self.buy()
        elif self.fast_sma[0] < self.slow_sma[0] and self.position:
            self.sell()

# class CustomStrategy(bt.Strategy):
#     params = (
#         ('rsi_period', 14),
#         ('macd_short', 12),
#         ('macd_long', 26),
#         ('macd_signal', 9),
#         ('bbands_period', 20),
#         ('bbands_devfactor', 2),
#     )

#     def __init__(self):
#         self.rsi = bt.indicators.RSI_SMA(self.data.close, period=self.params.rsi_period)
#         self.macd = bt.indicators.MACD(self.data.close, period_me1=self.params.macd_short, period_me2=self.params.macd_long, period_signal=self.params.macd_signal)
#         self.bbands = bt.indicators.BollingerBands(self.data.close, period=self.params.bbands_period, devfactor=self.params.bbands_devfactor)

#     def next(self):
#         # Buy conditions
#         if (
#             self.rsi < 30 and
#             self.macd.macd > self.macd.signal and
#             self.data.close < self.bbands.lines.bot
#         ):
#             self.buy()

#         # Sell conditions
#         elif (
#             self.rsi > 70 and
#             self.macd.macd < self.macd.signal and
#             self.data.close > self.bbands.lines.top
#         ):
#             self.sell()

# Initialize the Cerebro engine
cerebro = bt.Cerebro()

# Add the custom strategy to the Cerebro engine
cerebro.addstrategy(SimpleMovingAverageCross)

# Create a data feed from the dataset
data = CustomPandasData(dataname=dataset[dataset['ticker'] == 'AAPL'])

# Add the data feed to the Cerebro engine
cerebro.adddata(data)

# Set the initial cash balance
cerebro.broker.setcash(10000.0)

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
results = cerebro.run()

result = results[0]

# Analyze the performance
num_trades = len(result._tradespending)
num_wins = sum(1 for trade in result._tradespending if trade.pnl > 0)
num_losses = sum(1 for trade in result._tradespending if trade.pnl < 0)
winning_rate = num_wins / num_trades * 100 if num_trades > 0 else 0

# Print the performance metrics
print('Starting Portfolio Value: %.2f' % cerebro.broker.startingcash)
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Number of Trades: %d' % num_trades)
print('Winning Trades: %d' % num_wins)
print('Losing Trades: %d' % num_losses)
print('Winning Rate: %.2f%%' % winning_rate)

# Plot the result
#cerebro.plot()

Starting Portfolio Value: 10000.00
Starting Portfolio Value: 10000.00
Final Portfolio Value: nan
Number of Trades: 0
Winning Trades: 0
Losing Trades: 0
Winning Rate: 0.00%


In [38]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

# Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')

# Select a subset of columns
dataset = dataset.loc[idx[:, '2021-02-15':'2023-02-15'], ['adjusted_close', 'volume']]

# Assign names to the index
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

# Convert the 'date' column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

class CustomPandasData(bt.feeds.PandasData):
    lines = ('close', 'volume')
    params = (
        ('datetime', 'date'),
        ('open', None),
        ('high', None),
        ('low', None),
        ('close', 'adjusted_close'),
        ('volume', 'volume'),
        ('openinterest', None)
    )

class MovingAverageStrategy(bt.Strategy):
    params = (('fast', 10), ('slow', 50))

    def __init__(self):
        self.fast_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast)
        self.slow_ma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow)
        self.crossover = bt.indicators.CrossOver(self.fast_ma, self.slow_ma)

    def next(self):
        print('Close:', self.data.close[0])
        print('Fast MA:', self.fast_ma[0])
        print('Slow MA:', self.slow_ma[0])
        print('Crossover:', self.crossover[0])
        if not self.position:
            if self.crossover > 0:
                print('Buy order executed')
                self.buy()
        elif self.crossover < 0:
            print('Sell order executed')
            self.close()
        print('Portfolio Value:', self.broker.getvalue())

# Initialize the cerebro engine
cerebro = bt.Cerebro()

# Set the data
data = CustomPandasData(dataname=dataset)

# Add the data to cerebro
cerebro.adddata(data)

# Add the strategy
cerebro.addstrategy(MovingAverageStrategy)

# Set the starting cash
cerebro.broker.setcash(100000.0)

# Set the commission
cerebro.broker.setcommission(commission=0.001)

# Add observers for trade and portfolio performance metrics
cerebro.addobserver(bt.observers.Broker)
cerebro.addobserver(bt.observers.Trades)
cerebro.addobserver(bt.observers.DrawDown)
cerebro.addobserver(bt.observers.BuySell)

# Print the starting portfolio value
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run the backtest
cerebro.run()

# Print the final portfolio value
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Plot the results
#cerebro.plot()

Starting Portfolio Value: 100000.00


In [ ]:
import pandas as pd
from pandas import IndexSlice as idx
import backtrader as bt

# Load the dataset
dataset = pd.read_hdf('/home/groovyjac/projects/autonomous-portfolio-management/main_data_store_JDKv1.h5', 'stocks/prices/daily')

# Select a subset of columns
dataset = dataset.loc[idx[:, '2005-02-15':'2023-02-15'], ['adjusted_close', 'volume']]

# Assign names to the index
dataset.index.names = ['ticker', 'date']

# Reset the index
dataset = dataset.reset_index()

# Convert the 'date' column to datetime
dataset['date'] = pd.to_datetime(dataset['date'])

class CustomPandasData(bt.feeds.PandasData):
    lines = ('close', 'volume')
    params = (
        ('datetime', 'date'),
        ('open', None),
        ('high', None),
        ('low', None),
        ('close', 'adjusted_close'),
        ('volume', 'volume'),
        ('openinterest', None)
    )

# Unit Tests